# 📋 초기 설정 및 해독표 정리

> **마스터 매핑 테이블을 분석하여 설문 요약본과 패널 정보 해독표를 생성하는 프로세스**

---

## 🎯 프로젝트 개요

| 항목 | 내용 |
|------|------|
| **목적** | 마스터 매핑 테이블을 분석하여 구조화된 해독표 생성 |
| **입력** | `../data/processed/00_master_mapping_table.csv` |
| **출력** | 설문 요약본 + 패널 정보 해독표 |
| **결과** | 분석에 필요한 구조화된 메타데이터 제공 |

## 📋 처리 과정

### 🔹 **설문 요약본 (Quickpoll Summary)**
- **대상**: Quickpoll 파일에서 추출된 설문 정보
- **내용**: 설문 제목, 참여자 수, 설문 기간 등
- **출력**: `quickpoll_summary.csv`

### 🔹 **패널 정보 해독표 (Panel Data Dictionary)**
- **대상**: Welcome 파일에서 추출된 패널 정보
- **내용**: 변수명, 설명, 타입, 코드-라벨 매핑
- **출력**: `panel_data_dictionary.csv`

---

## 📁 노트북 구조

1. **라이브러리 설정** - 필요한 패키지 import
2. **마스터 매핑 테이블 로드** - 원본 데이터 불러오기
3. **설문 요약본 생성** - Quickpoll 데이터 정리
4. **패널 정보 해독표 생성** - Welcome 데이터 정리
5. **결과 확인** - 생성된 파일들 검증

## 📚 1. 라이브러리 설정

데이터 처리에 필요한 라이브러리들을 import합니다.

In [ ]:
import pandas as pd
import numpy as np
import os
import glob

## 📁 2. 마스터 매핑 테이블 로드

1단계에서 생성된 마스터 매핑 테이블을 불러옵니다.

In [ ]:
# 1. 원본 마스터 매핑 테이블 불러오기
# 파일 경로를 사용자 환경에 맞게 확인해주세요.
try:
    master_df = pd.read_csv('../data/processed/00_master_mapping_table.csv', encoding='utf-8')
    print("✅ UTF-8 인코딩으로 파일을 성공적으로 불러왔습니다.")
except UnicodeDecodeError:
    master_df = pd.read_csv('../data/processed/00_master_mapping_table.csv', encoding='cp949')
    print("✅ CP949 인코딩으로 파일을 성공적으로 불러왔습니다.")

print(f"\n📊 마스터 매핑 테이블 정보:")
print(f"- 데이터 크기: {master_df.shape}")
print(f"- 컬럼 수: {len(master_df.columns)}")
print(f"- 출처 파일 수: {master_df['source_file'].nunique()}")

print(f"\n📋 컬럼 목록:")
for i, col in enumerate(master_df.columns, 1):
    print(f"{i:2d}. {col}")

print(f"\n📁 출처 파일별 데이터 수:")
print(master_df['source_file'].value_counts())

## 📊 3. 설문 요약본 생성

Quickpoll 파일에서 추출된 설문 정보를 정리하여 설문 요약본을 생성합니다.

In [ ]:
# 2. '설문 요약본 (Quickpoll Summary)' 추출 및 정리
# 'source_file' 컬럼에 'qpoll'이 포함된 행만 필터링합니다.
print("=== 설문 요약본 생성 시작 ===")
quickpoll_df = master_df[master_df['source_file'].str.contains('qpoll', na=False)].copy()
print(f"Quickpoll 관련 데이터 수: {len(quickpoll_df)}개")

# 불필요한 컬럼 및 중간에 반복되는 헤더 행 제거
columns_to_drop = ['Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', '변수명', '문항', '문항유형']
existing_columns_to_drop = [col for col in columns_to_drop if col in quickpoll_df.columns]
quickpoll_df = quickpoll_df.drop(columns=existing_columns_to_drop)

# 빈 값과 중복 헤더 제거
quickpoll_df = quickpoll_df.dropna(subset=['설문제목'])
quickpoll_df = quickpoll_df[quickpoll_df['설문제목'] != '설문제목']

print(f"정리 후 데이터 수: {len(quickpoll_df)}개")

# 데이터 타입 정리 (숫자여야 하는 컬럼들을 숫자로 변경)
count_cols = [col for col in quickpoll_df.columns if '_CNT' in col or '총참여자수' in col]
print(f"숫자 컬럼 변환: {count_cols}")

for col in count_cols:
    if col in quickpoll_df.columns:
        quickpoll_df[col] = pd.to_numeric(quickpoll_df[col], errors='coerce')
        print(f"  - {col}: 변환 완료")

# 결과 저장
quickpoll_output_path = '../data/processed/quickpoll_summary.csv'
quickpoll_df.to_csv(quickpoll_output_path, index=False, encoding='utf-8-sig')

print(f"\n✅ 설문 요약본 저장 완료: {quickpoll_output_path}")
print(f"📊 최종 데이터 크기: {quickpoll_df.shape}")

# 데이터 미리보기
print(f"\n📋 설문 요약본 미리보기:")
display(quickpoll_df.head())

## 👥 4. 패널 정보 해독표 생성

Welcome 파일에서 추출된 패널 정보를 정리하여 패널 정보 해독표를 생성합니다.

In [ ]:
# 3. '패널 정보 해독표 (Panel Data Dictionary)' 추출 및 정리
# 'source_file' 컬럼에 'Welcome'이 포함된 행만 필터링합니다.
print("=== 패널 정보 해독표 생성 시작 ===")
panel_df = master_df[master_df['source_file'].str.contains('Welcome', na=False)].copy()
print(f"Welcome 관련 데이터 수: {len(panel_df)}개")

# 필요한 컬럼만 선택하고, 이름 변경
required_columns = ['변수명', '문항', '문항유형', '보기1', '보기2']
existing_columns = [col for col in required_columns if col in panel_df.columns]
panel_df = panel_df[existing_columns]

# 컬럼명 변경
column_mapping = {
    '변수명': 'variable',
    '문항': 'description', 
    '문항유형': 'type',
    '보기1': 'code',
    '보기2': 'label'
}
panel_df = panel_df.rename(columns=column_mapping)

print(f"사용 가능한 컬럼: {list(panel_df.columns)}")

# 각 변수(variable)의 설명(description)과 타입(type)은 첫 행에만 있으므로, ffill()로 채워줍니다.
if 'variable' in panel_df.columns:
    panel_df['variable'] = panel_df['variable'].fillna(method='ffill')
if 'description' in panel_df.columns:
    panel_df['description'] = panel_df['description'].fillna(method='ffill')
if 'type' in panel_df.columns:
    panel_df['type'] = panel_df['type'].fillna(method='ffill')

# 실제 코드-라벨 값이 있는 행만 남깁니다. (변수 설명이 적힌 행은 제외)
if 'code' in panel_df.columns:
    panel_df = panel_df.dropna(subset=['code'])

print(f"정리 후 데이터 수: {len(panel_df)}개")

# 결과 저장
panel_dict_output_path = '../data/processed/panel_data_dictionary.csv'
panel_df.to_csv(panel_dict_output_path, index=False, encoding='utf-8-sig')

print(f"\n✅ 패널 정보 해독표 저장 완료: {panel_dict_output_path}")
print(f"📊 최종 데이터 크기: {panel_df.shape}")

# 데이터 미리보기
print(f"\n📋 패널 정보 해독표 미리보기:")
display(panel_df.head())

## ✅ 5. 결과 확인

생성된 파일들을 확인하고 최종 결과를 검증합니다.

In [ ]:
# 생성된 파일들 확인
print("=== 생성된 파일 확인 ===")

# 설문 요약본 확인
if os.path.exists('../data/processed/quickpoll_summary.csv'):
    quickpoll_check = pd.read_csv('../data/processed/quickpoll_summary.csv')
    print(f"✅ 설문 요약본: {quickpoll_check.shape} (행, 열)")
    print(f"   - 컬럼 수: {len(quickpoll_check.columns)}")
    print(f"   - 파일 크기: {os.path.getsize('../data/processed/quickpoll_summary.csv')} bytes")
else:
    print("❌ 설문 요약본 파일을 찾을 수 없습니다.")

# 패널 정보 해독표 확인
if os.path.exists('../data/processed/panel_data_dictionary.csv'):
    panel_check = pd.read_csv('../data/processed/panel_data_dictionary.csv')
    print(f"✅ 패널 정보 해독표: {panel_check.shape} (행, 열)")
    print(f"   - 컬럼 수: {len(panel_check.columns)}")
    print(f"   - 파일 크기: {os.path.getsize('../data/processed/panel_data_dictionary.csv')} bytes")
else:
    print("❌ 패널 정보 해독표 파일을 찾을 수 없습니다.")

print(f"\n🎉 해독표 정리 완료!")
print(f"📁 생성된 파일들:")
print(f"   1. quickpoll_summary.csv - 설문 요약본")
print(f"   2. panel_data_dictionary.csv - 패널 정보 해독표")
print(f"\n💡 이제 이 해독표들을 활용하여 데이터 분석을 진행할 수 있습니다!")